In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from scipy import stats
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV


In [9]:
df = pd.read_csv("train.csv")
new = df.sample(int(len(df) * 0.3))
new = new.dropna()
new

,customer_id,customer_visit_score,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
9871,csid_9872,6.013294,4.668033,0.953059,2.127852,15.827250,14.334730,1.277738,-0.164635,D,F,1
1809,csid_1810,19.892049,2.803124,0.219727,0.235768,0.470501,4.632321,2.223789,6.670960,C,BA,0
6482,csid_6483,33.575818,7.075375,0.009457,-0.019191,2.237958,6.347256,1.824649,3.569503,A,BA,0
2059,csid_2060,14.527294,5.746693,0.103823,-0.048117,0.202317,3.329421,3.288081,54.665819,B,AA,0
172,csid_173,26.915048,3.763770,-0.053293,0.044950,0.921721,4.947823,3.277692,11.886612,C,BA,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8988,csid_8989,18.644053,5.764087,-0.201803,-0.058028,17.512131,14.551194,1.590037,-0.193948,B,BA,0
6555,csid_6556,24.509638,2.536657,0.188774,0.373966,0.243767,3.989264,3.280461,22.941953,B,A,0
6082,csid_6083,17.418433,5.354174,0.013164,0.002439,0.515487,3.658234,4.151880,9.187190,B,BA,0
3527,csid_3528,16.581529,6.201418,-0.022219,-0.046312,0.320332,4.354506,3.470028,18.981165,C,A,0


In [14]:
ncols = new.columns[::-3]
new[ncols] = new[ncols].apply(lambda x: (x-x.min())/(x.max()-x.min())) #normalizing
norm = new.drop(["customer_id"], axis=1)
norm


,customer_visit_score,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
9871,0.115063,0.320900,0.465086,0.196700,0.340290,0.742372,0.091924,0.001490,D,F,1.0
1809,0.412491,0.194024,0.237870,0.048858,0.009419,0.118583,0.213269,0.034490,C,BA,0.0
6482,0.705741,0.484680,0.172720,0.028936,0.047500,0.228840,0.162073,0.019517,A,BA,0.0
2059,0.297522,0.394285,0.201958,0.026676,0.003641,0.034817,0.349782,0.266189,B,AA,0.0
172,0.562997,0.259380,0.153277,0.033948,0.019141,0.138867,0.348449,0.059669,C,BA,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8988,0.385746,0.395469,0.107263,0.025902,0.376591,0.756289,0.131981,0.001349,B,BA,0.0
6555,0.511448,0.175895,0.228279,0.059656,0.004534,0.077240,0.348804,0.113039,B,A,0.0
6082,0.359480,0.367581,0.173868,0.030626,0.010389,0.055957,0.460577,0.046637,B,BA,0.0
3527,0.341545,0.425222,0.162905,0.026817,0.006184,0.100722,0.373119,0.093918,C,A,0.0


In [15]:
from sklearn.preprocessing import LabelEncoder#encoding 
cols = norm.columns[-3::]
le = LabelEncoder()
norm[cols] = norm[cols].apply(lambda col: le.fit_transform(col))
enc=norm[norm.columns[1::]]
enc

,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
9871,0.320900,0.465086,0.196700,0.340290,0.742372,0.091924,0.001490,4,4,1
1809,0.194024,0.237870,0.048858,0.009419,0.118583,0.213269,0.034490,3,2,0
6482,0.484680,0.172720,0.028936,0.047500,0.228840,0.162073,0.019517,0,2,0
2059,0.394285,0.201958,0.026676,0.003641,0.034817,0.349782,0.266189,2,1,0
172,0.259380,0.153277,0.033948,0.019141,0.138867,0.348449,0.059669,3,2,0
...,...,...,...,...,...,...,...,...,...,...
8988,0.395469,0.107263,0.025902,0.376591,0.756289,0.131981,0.001349,2,2,0
6555,0.175895,0.228279,0.059656,0.004534,0.077240,0.348804,0.113039,2,0,0
6082,0.367581,0.173868,0.030626,0.010389,0.055957,0.460577,0.046637,2,2,0
3527,0.425222,0.162905,0.026817,0.006184,0.100722,0.373119,0.093918,3,0,0


In [17]:
wo = enc.copy()#removing outliers
wo=wo[['customer_product_search_score', 'customer_ctr_score', 'customer_stay_score', 'customer_frequency_score', 'customer_product_variation_score', 'customer_order_score', 'customer_affinity_score','customer_active_segment','X1','customer_category']]
wo = wo[(np.abs(stats.zscore(wo)) < 3).all(axis=1)]
wo

,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
1809,0.194024,0.237870,0.048858,0.009419,0.118583,0.213269,0.034490,3,2,0
6482,0.484680,0.172720,0.028936,0.047500,0.228840,0.162073,0.019517,0,2,0
2059,0.394285,0.201958,0.026676,0.003641,0.034817,0.349782,0.266189,2,1,0
172,0.259380,0.153277,0.033948,0.019141,0.138867,0.348449,0.059669,3,2,0
3974,0.512379,0.171695,0.029787,0.068733,0.419588,0.230541,0.005860,3,2,0
...,...,...,...,...,...,...,...,...,...,...
776,0.321385,0.194691,0.030022,0.007915,0.204887,0.256461,0.028687,2,2,0
6555,0.175895,0.228279,0.059656,0.004534,0.077240,0.348804,0.113039,2,0,0
6082,0.367581,0.173868,0.030626,0.010389,0.055957,0.460577,0.046637,2,2,0
3527,0.425222,0.162905,0.026817,0.006184,0.100722,0.373119,0.093918,3,0,0


In [18]:
X = wo[wo.columns[:-1]]
y = wo['customer_category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [19]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
nb = GaussianNB()
nb.fit(X_train, y_train)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
svm = SVC(kernel="linear", probability=True)
svm.fit(X_train, y_train)
log = LogisticRegression()
log.fit(X_train, y_train)



LogisticRegression()

In [21]:
a = [
    ["Random Forest ", clf],["K neighbors ", knn], ["Naive Bayes ", nb],  ["Support vector machine ", svm],  ["Regression ", log]
]
pred_v = []
auc_v = []
for algorithm in a:
    algorithm[1].fit(X_train, y_train)
    pred = algorithm[1].predict(X_test)
    proba = algorithm[1].predict_proba(X_test)[:, 1]
    pred_v.append([algorithm[0], pred])
    auc = metrics.roc_auc_score(y_test, proba)
    auc_v.append([algorithm[0], auc])
print("Accuracy for each algorithm:\n")
for value in pred_v:
    print(value[0], metrics.accuracy_score(y_test, value[1]))
print("AUC:\n")
for value in auc_v:
    print(*value)



Accuracy for each algorithm:

Random Forest  0.9811764705882353
K neighbors  0.971764705882353
Naive Bayes  0.9435294117647058
Support vector machine  0.9776470588235294
Regression  0.9705882352941176
AUC:

Random Forest  0.9217634652822844
K neighbors  0.9316055591607182
Naive Bayes  0.9366077222582738
Support vector machine  0.9655256327060351
Regression  0.9459766385463984


In [22]:
print("Random forest showed itself as the best classifier with the highest accuracy")

Random forest showed itself as the best classifier with the highest accuracy
